In [5]:
from email.mime.multipart import MIMEMultipart
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email import encoders
import smtplib   
import smtplib
from google.cloud import storage
import json
#import sendgrid
import os
#from sendgrid.helpers.mail import *

In [13]:
to='motilalmeher9@gmail.com'
fromemail='motilalmeher11@gmail.com'
password='Motiminku69#'

   
def sendmail(body,subject,file,path,bucket):
    msg = MIMEMultipart()
    msg["From"] = fromemail
    msg["To"] = to
    msg["Subject"] = subject

    #bucket = self.client.get_bucket(bucket)
    #blob = storage.Blob(path+file, bucket)
    #content = blob.download_as_string()

    maintype='application'
    subtype='vnd.ms-excel'
    #attachment = MIMEBase(maintype, subtype)
    #attachment.set_payload(content)
    #encoders.encode_base64(attachment)
    #attachment.add_header("Content-Disposition", "attachment", filename=file)
    msg.attach(body)
    #msg.attach(attachment)

    mailserver = smtplib.SMTP('smtp.gmail.com:587')
    mailserver.ehlo()
    mailserver.starttls()
    mailserver.ehlo()  #again
    mailserver.login(fromemail,password)
    mailserver.sendmail(fromemail,to, msg.as_string())
    mailserver.quit()

In [14]:
sendmail('hello','subject','file','path','bucket')

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials k20sm14864307pfa.34 - gsmtp')

In [29]:
import os
import pandas as pd
from googleapiclient import discovery
import json
model_name='chrun_1'

In [30]:
service = discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format('projectmtna', model_name)

In [31]:
df=pd.read_csv("gs://cvm_data/datasets/testing_process/data.csv")

In [32]:
len(df.columns)

143

In [67]:
X_validation=df.drop(columns=['mobile_number','churn'])

In [68]:
len(X_validation.columns)

141

In [75]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [102]:
x=[]
for index, row in X_validation.head(20).iterrows():
    x.append(list(row.values))

In [77]:
def predict(model_name, instances):
    
    service = discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format('projectmtna', model_name)
    

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
    
    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [81]:
predictions=predict(model_name,x)

In [89]:
rows = []

for idx, pred in enumerate(predictions):
    rows.append((int(df['mobile_number'][idx]), pred))

In [90]:
rows

[(7000436570, 1.0),
 (7001897999, 0.0),
 (7001922492, 0.0),
 (7000539299, 0.0),
 (7001492962, 0.0)]

In [70]:
response = service.projects().predict(
        name=name,
        body={'instances':x}
    ).execute()

In [91]:
response = service.projects().predict_proba(
        name=name,
        body={'instances':x}
    ).execute()

AttributeError: 'Resource' object has no attribute 'predict_proba'

In [76]:
!gcloud ai-platform predict \
--model $model_name \
--json-instances $input_file\
--region global

Using endpoint [https://ml.googleapis.com/]
[1.0, 0.0, 0.0, 0.0, 0.0]


In [71]:
response

{'predictions': [1.0, 0.0, 0.0, 0.0, 0.0]}

In [95]:
from google.cloud import storage
from sklearn.externals import joblib

bucket_name = "cvm_data"
gs_model = "jobdir/1ec9256e-d93b-4eb5-a6a0-b0fa69b8c797/model.pkl"  # path in your Cloud Storage bucket
local_model = "model.pkl"  # path in your local machine

client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(gs_model)
blob.download_to_filename(local_model)

In [96]:
model = joblib.load(local_model)
model.predict_proba(x)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/base.py:297: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


array([[0.00000000e+000, 1.00000000e+000],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 2.84768029e-292],
       [1.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 0.00000000e+000]])

In [104]:
model.predict(x)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [103]:
model.score(x,df.head(20)['churn'])

0.85

In [105]:
df.head(20)['churn']

0     1.0
1     0.0
2     1.0
3     1.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: churn, dtype: float64